In [1]:
import pylab
import numpy as np 
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import csv
from bs4 import BeautifulSoup  
from wordcloud import WordCloud,STOPWORDS

import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list

import operator
import re

pd.options.mode.chained_assignment = None  # default='warn'

print "initial imports complete"

/Applications/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


initial imports complete


In [2]:
from bokeh.io import output_notebook, show
output_notebook()

from bokeh.plotting import figure
#p = figure(plot_width=400, plot_height=400)
#p.square([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=[10, 15, 20, 25, 30], color="firebrick", alpha=0.6)
#p.line(x, y, line_width=2)
#show(p) # show the results

from bokeh.charts import Scatter
#p = Scatter(flowers, x='petal_length', y='petal_width', color='species', legend='top_left')
#show(p)

from bokeh.charts import Bar
#p = Bar(autompg, label='yr', values='mpg', agg='median', group='origin', 
#        title="Median MPG by YR, grouped by ORIGIN", legend='top_left', tools='crosshair')
#show(p)

print "bokeh imports complete"

Loading BokehJS ...

bokeh imports complete


In [5]:
reviews = pd.read_pickle('reviews_cleaned')
reviews.head(5)

,user_id,votes.cool_x,business_id,votes.funny_x,stars,date,type_x,votes.useful_x,return_visit,first_visit,...,votes.funny_y,compliments.photos,compliments.funny,votes.useful_y,friends2,nfriends,elite2,nelite,text_clean,text_length
0,PUFPaY9KxDAcGqfsorJp3Q,0,5UmKMjUEUNdYWqANhGckJw,0,4,2012-08-01,review,0,False,False,...,25,NaN,NaN,166,[[]],1,[[]],1,mr hoagie institution walking seem like throwb...,272
1,PUFPaY9KxDAcGqfsorJp3Q,0,WaHXyBwljbKNPmmJZn5j8Q,0,3,2012-08-01,review,0,False,False,...,25,NaN,NaN,166,[[]],1,[[]],1,shop save,9
2,PUFPaY9KxDAcGqfsorJp3Q,0,u22QDAON1kk0wjoTliyftw,0,4,2012-08-05,review,4,False,False,...,25,NaN,NaN,166,[[]],1,[[]],1,admit first time hate absolutely hated pizza t...,484
3,PUFPaY9KxDAcGqfsorJp3Q,0,4ykgzzzGEWjMD5lwk1-l9A,0,3,2009-05-20,review,0,False,False,...,25,NaN,NaN,166,[[]],1,[[]],1,mixed results would definitely recommend buyin...,102
4,PUFPaY9KxDAcGqfsorJp3Q,0,2PfavOTufsPCRdYm-bFcpw,0,5,2012-08-01,review,0,False,False,...,25,NaN,NaN,166,[[]],1,[[]],1,place great ice cream shop favorite birch beer...,142


In [4]:
#Now pick out just those two sub-sets of reviews (first-visits and return visits)
reviewsR = reviews[reviews.return_visit == True]
print(reviewsR.shape)

reviewsF = reviews[reviews.first_visit == True]
print(reviewsF.shape)

#For comparison, also pick out reviews with NO return visits
reviewsN = reviews[reviews.return_visit == False]
reviewsN = reviewsN[reviewsN.first_visit == False]
print(reviewsN.shape)

#Pick out GOOD return reviews
reviewsRG = reviewsR[reviewsR.stars == 5]
print(reviewsRG.shape)

#Pick out the BAD return reviews
reviewsRB = reviewsR[reviewsR.stars == 1]
print(reviewsRB.shape)

(69792, 41)
(69792, 41)
(2093827, 41)
(30214, 41)
(10244, 41)


In [5]:
star5 = reviews[reviews['stars'] == 5]
star4 = reviews[reviews['stars'] == 4]
star3 = reviews[reviews['stars'] == 3]
star2 = reviews[reviews['stars'] == 2]
star1 = reviews[reviews['stars'] == 1]

star5R = reviewsR[reviewsR['stars'] == 5]
star4R = reviewsR[reviewsR['stars'] == 4]
star3R = reviewsR[reviewsR['stars'] == 3]
star2R = reviewsR[reviewsR['stars'] == 2]
star1R = reviewsR[reviewsR['stars'] == 1]

star5F = reviewsF[reviewsF['stars'] == 5]
star4F = reviewsF[reviewsF['stars'] == 4]
star3F = reviewsF[reviewsF['stars'] == 3]
star2F = reviewsF[reviewsF['stars'] == 2]
star1F = reviewsF[reviewsF['stars'] == 1]

In [6]:
m5 = star5['text_length'].mean()
m4 = star4['text_length'].mean()
m3 = star3['text_length'].mean()
m2 = star2['text_length'].mean()
m1 = star1['text_length'].mean()

s5 = star5['text_length'].std()
s4 = star4['text_length'].std()
s3 = star3['text_length'].std()
s2 = star2['text_length'].std()
s1 = star1['text_length'].std()

m5R = star5R['text_length'].mean()
m4R = star4R['text_length'].mean()
m3R = star3R['text_length'].mean()
m2R = star2R['text_length'].mean()
m1R = star1R['text_length'].mean()

s5R = star5R['text_length'].std()
s4R = star4R['text_length'].std()
s3R = star3R['text_length'].std()
s2R = star2R['text_length'].std()
s1R = star1R['text_length'].std()

m5F = star5F['text_length'].mean()
m4F = star4F['text_length'].mean()
m3F = star3F['text_length'].mean()
m2F = star2F['text_length'].mean()
m1F = star1F['text_length'].mean()

s5F = star5F['text_length'].std()
s4F = star4F['text_length'].std()
s3F = star3F['text_length'].std()
s2F = star2F['text_length'].std()
s1F = star1F['text_length'].std()

In [16]:
p = figure(plot_width=400, plot_height=400)
p.square([0, 1, 2, 3, 4, 5, 6], [200, m1, m2, m3, m4, m5, 900], size=[0, s1/10, s2/10, s3/10, s4/10, s5/10, 0], legend="All reviews", color="black", alpha=0.6)
p.square([0, 1, 2, 3, 4, 5, 6], [m1F, m1F, m2F, m3F, m4F, m5F, m1F], size=[0, s1F/10, s2F/10, s3F/10, s4F/10, s5F/10, 0], legend="First-time reviews", color="red", alpha=0.6)
p.square([0, 1, 2, 3, 4, 5, 6], [m1R, m1R, m2R, m3R, m4R, m5R, m1R], size=[0, s1R/10, s2R/10, s3R/10, s4R/10, s5R/10, 0], legend="Return reviews", color="blue", alpha=0.6)
p.xaxis.axis_label = "Star rating (out of 5)"
p.yaxis.axis_label = "Average length of review in characters"
p.legend.location = "top_left"
show(p) # show the results

In [17]:
reviews_cleanN=[]
nrev = reviewsF['text_clean'].size
for i in range( 0, nrev ):
    reviews_cleanN.append(reviewsN['text_clean'].iloc[i])
    if(i%1000 == 0):
        print("Appended %d of %d so far\n" %(i, nrev))

reviews_cleanF=[]
nrev = reviewsF['text_clean'].size
for i in range( 0, nrev ):
    reviews_cleanF.append(reviewsF['text_clean'].iloc[i])
    if(i%1000 == 0):
        print("Appended %d of %d so far\n" %(i, nrev))
        
reviews_cleanR=[]
nrev = reviewsR['text_clean'].size
for i in range( 0, nrev ):
    reviews_cleanR.append(reviewsR['text_clean'].iloc[i])
    if(i%1000 == 0):
        print("Appended %d of %d so far\n" %(i, nrev))
        
reviews_cleanRG=[]
nrev = reviewsRG['text_clean'].size
for i in range( 0, nrev ):
    reviews_cleanRG.append(reviewsRG['text_clean'].iloc[i])
    if(i%1000 == 0):
        print("Appended %d of %d so far\n" %(i, nrev))
        
reviews_cleanRB=[]
nrev = reviewsRB['text_clean'].size
for i in range( 0, nrev ):
    reviews_cleanRB.append(reviewsRB['text_clean'].iloc[i])
    if(i%1000 == 0):
        print("Appended %d of %d so far\n" %(i, nrev))

Appended 0 of 69792 so far

Appended 1000 of 69792 so far

Appended 2000 of 69792 so far

Appended 3000 of 69792 so far

Appended 4000 of 69792 so far

Appended 5000 of 69792 so far

Appended 6000 of 69792 so far

Appended 7000 of 69792 so far

Appended 8000 of 69792 so far

Appended 9000 of 69792 so far

Appended 10000 of 69792 so far

Appended 11000 of 69792 so far

Appended 12000 of 69792 so far

Appended 13000 of 69792 so far

Appended 14000 of 69792 so far

Appended 15000 of 69792 so far

Appended 16000 of 69792 so far

Appended 17000 of 69792 so far

Appended 18000 of 69792 so far

Appended 19000 of 69792 so far

Appended 20000 of 69792 so far

Appended 21000 of 69792 so far

Appended 22000 of 69792 so far

Appended 23000 of 69792 so far

Appended 24000 of 69792 so far

Appended 25000 of 69792 so far

Appended 26000 of 69792 so far

Appended 27000 of 69792 so far

Appended 28000 of 69792 so far

Appended 29000 of 69792 so far

Appended 30000 of 69792 so far

Appended 31000 of 697

In [18]:
mystringN = ''.join(reviews_cleanN)
mystringF = ''.join(reviews_cleanF)
mystringR = ''.join(reviews_cleanR)
mystringRG = ''.join(reviews_cleanRG)
mystringRB = ''.join(reviews_cleanRB)

In [19]:
from nltk.corpus import stopwords # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
reviews_total = []
reviews_total.append(mystringN)
reviews_total.append(mystringF)
reviews_total.append(mystringR)
reviews_total.append(mystringRG)
reviews_total.append(mystringRB)

In [21]:
transformer = TfidfTransformer() #to do tfidf weighting on all of the bag_of_words vectors

reviews_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500, ngram_range=(1,1)) 
reviews_features = reviews_vec.fit_transform(reviews_total)
reviews_tfidf = transformer.fit_transform(reviews_features)
reviews_features = reviews_features.toarray()
reviews_tfidf = reviews_tfidf.toarray()


reviews2_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500, ngram_range=(2,2)) 
reviews2_features = reviews2_vec.fit_transform(reviews_total)
reviews2_tfidf = transformer.fit_transform(reviews2_features)
reviews2_features = reviews2_features.toarray()
reviews2_tfidf = reviews2_tfidf.toarray()


reviews3_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500, ngram_range=(3,3)) 
reviews3_features = reviews3_vec.fit_transform(reviews_total)
reviews3_tfidf = transformer.fit_transform(reviews3_features)
reviews3_features = reviews3_features.toarray()
reviews3_tfidf = reviews3_tfidf.toarray()

In [22]:
vocab = reviews_vec.get_feature_names()
vocab2 = reviews2_vec.get_feature_names()
vocab3 = reviews3_vec.get_feature_names()

print len(vocab)
#print (vocab)

# Sum up the counts of each vocabulary word
reviews_vec_total = np.sum(reviews_features, axis=0)
#print reviews_vec_total
#print reviews_tfidf

500


In [23]:
#mystringN [0], mystringF [1], mystringR [2], mystringRG [3],  mystringRB[4] 

i=0
dist = reviews_tfidf[i] #one row of word features
count_dict = dict(zip(vocab, dist))
sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
word, freq = zip(*sorted_count_dict)

dist2 = reviews2_tfidf[i] #one row of word features
count_dict2 = dict(zip(vocab2, dist2))
sorted_count_dict2 = sorted(count_dict2.items(), key=operator.itemgetter(1),reverse=True)
word2, freq2 = zip(*sorted_count_dict2)

dist3 = reviews3_tfidf[i] #one row of word features
count_dict3 = dict(zip(vocab3, dist3))
sorted_count_dict3 = sorted(count_dict3.items(), key=operator.itemgetter(1),reverse=True)
word3, freq3 = zip(*sorted_count_dict3)

for i in range (0,100):
    print i+1, map(lambda x: x.encode('ascii'), word[i:i+1]), map(lambda x: x.encode('ascii'), word2[i:i+1]), map(lambda x: x.encode('ascii'), word3[i:i+1])

1 ['good'] ['go back'] ['sweet potato fries']
2 ['food'] ['pretty good'] ['would go back']
3 ['place'] ['ice cream'] ['wait go back']
4 ['like'] ['really good'] ['go back try']
5 ['great'] ['first time'] ['definitely go back']
6 ['one'] ['happy hour'] ['food pretty good']
7 ['get'] ['next time'] ['nothing write home']
8 ['really'] ['even though'] ['come back try']
9 ['time'] ['come back'] ['would definitely recommend']
10 ['would'] ['food good'] ['would highly recommend']
11 ['go'] ['mac cheese'] ['every time go']
12 ['also'] ['great place'] ['french onion soup']
13 ['service'] ['feel like'] ['within walking distance']
14 ['back'] ['every time'] ['bottom line would']
15 ['nice'] ['make sure'] ['would come back']
16 ['little'] ['one best'] ['really really good']
17 ['well'] ['highly recommend'] ['definitely come back']
18 ['even'] ['good food'] ['would definitely go']
19 ['menu'] ['beer selection'] ['keep coming back']
20 ['got'] ['pretty much'] ['first time went']
21 ['bar'] ['much bet

In [63]:
i=4
dist = reviews_tfidf[i] #one row of word features
count_dict = dict(zip(vocab, dist))
sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
word, freq = zip(*sorted_count_dict)

dist2 = reviews2_tfidf[i] #one row of word features
count_dict2 = dict(zip(vocab2, dist2))
sorted_count_dict2 = sorted(count_dict2.items(), key=operator.itemgetter(1),reverse=True)
word2, freq2 = zip(*sorted_count_dict2)

dist3 = reviews3_tfidf[i] #one row of word features
count_dict3 = dict(zip(vocab3, dist3))
sorted_count_dict3 = sorted(count_dict3.items(), key=operator.itemgetter(1),reverse=True)
word3, freq3 = zip(*sorted_count_dict3)

x = word[0:99]
y = word2[0:99]
z = word3[0:99]



In [64]:
df_words = pd.DataFrame({'1-gram' : x,
                         '2-gram' : y, 
                         '3-gram' : z})
df_words.head(21)

,1-gram,2-gram,3-gram
0,would,customer service,never go back
1,time,last time,never come back
2,back,go back,poor customer service
3,get,come back,asked speak manager
4,one,first time,horrible customer service
5,place,even though,go somewhere else
6,like,said would,long story short
7,service,make sure,worst customer service
8,food,came back,lack customer service
9,said,went back,give another try


In [7]:
reviews_reduced = reviews.drop('name', axis=1)
reviews_reduced = reviews_reduced.drop('votes.funny_x', axis=1)
reviews_reduced = reviews_reduced.drop('votes.funny_y', axis=1)
reviews_reduced = reviews_reduced.drop('votes.cool_x', axis=1)
reviews_reduced = reviews_reduced.drop('votes.cool_y', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.photos', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.cute', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.hot', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.cool', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.more', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.list', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.note', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.plain', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.writer', axis=1)
reviews_reduced = reviews_reduced.drop('compliments.profile', axis=1)
reviews_reduced = reviews_reduced.drop('friends', axis=1)
reviews_reduced = reviews_reduced.drop('fans', axis=1)

In [10]:
reviews_reduced = reviews_reduced.drop('elite', axis=1)
reviews_reduced = reviews_reduced.drop('elite2', axis=1)
reviews_reduced = reviews_reduced.drop('friends2', axis=1)
reviews_reduced = reviews_reduced.drop('type_x', axis=1)
reviews_reduced = reviews_reduced.drop('type_y', axis=1)
reviews_reduced = reviews_reduced.drop('votes.useful_x', axis=1)
reviews_reduced = reviews_reduced.drop('votes.useful_y', axis=1)
reviews_reduced = reviews_reduced.drop('yelping_since', axis=1)

In [11]:
#reviews_reduced.to_pickle('reviews_reduced')

In [2]:
business = business = pd.read_csv("yelp_academic_dataset_business.csv", na_filter=False, index_col = 'business_id')

#business.to_pickle('business')
print business.index[2]
business_id_new = business.index
business['business_id_new'] = business_id_new

3eu6MEFlq2Dg7bQh8QbdOg


In [3]:
a = business['business_id_new']
b = business['name']
c = business['city']
d = business['state']
e = business['stars']

business_info = pd.DataFrame({'business_id' : a,
                         'business_name' : b, 
                         'city' : c,
                          'state' : d,
                           'business_stars' : e})

business_info.head(10)

,business_id,business_name,business_stars,city,state
business_id,,,,,
5UmKMjUEUNdYWqANhGckJw,5UmKMjUEUNdYWqANhGckJw,Mr Hoagie,4.5,Dravosburg,PA
UsFtqoBl7naz8AVUBZMjQQ,UsFtqoBl7naz8AVUBZMjQQ,Clancy's Pub,3.5,Dravosburg,PA
3eu6MEFlq2Dg7bQh8QbdOg,3eu6MEFlq2Dg7bQh8QbdOg,Joe Cislo's Auto,5.0,Dravosburg,PA
cE27W9VPgO88Qxe4ol6y_g,cE27W9VPgO88Qxe4ol6y_g,Cool Springs Golf Center,2.5,Bethel Park,PA
HZdLhv6COCleJMo7nPl-RA,HZdLhv6COCleJMo7nPl-RA,Verizon,2.5,Pittsburgh,PA
mVHrayjG3uZ_RLHkLj-AMg,mVHrayjG3uZ_RLHkLj-AMg,Emil's Lounge,5.0,rankin,PA
VHYT1qgQw5JvXAlUDhTArA,VHYT1qgQw5JvXAlUDhTArA,Extreme Fitness 2,2.5,Pittsburgh,PA
KayYbHCt-RkbGcPdGOThNg,KayYbHCt-RkbGcPdGOThNg,Alexion's Bar & Grill,4.0,Carnegie,PA
b12U9TFESStdy7CsTtcOeg,b12U9TFESStdy7CsTtcOeg,Flynn's Tire And Auto Service,2.5,Carnegie,PA


In [4]:
reviews = pd.read_pickle('reviews_reduced')

In [80]:
a = reviews['business_id']
b = reviews['date']
c = reviews['stars']
d = reviews['text_clean']
e = reviews['day']
f = reviews['month']
g = reviews['year']

reviews_info = pd.DataFrame({'business_id' : a,
                         'date' : b, 
                         'stars' : c,
                         'text_clean' : d,
                          'day' : e ,
                           'month' : f,
                           'year' : g})

reviews_info.head(10)

,business_id,date,day,month,stars,text_clean,year
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,1,8,4,mr hoagie institution walking seem like throwb...,2012
1,WaHXyBwljbKNPmmJZn5j8Q,2012-08-01,1,8,3,shop save,2012
2,u22QDAON1kk0wjoTliyftw,2012-08-05,5,8,4,admit first time hate absolutely hated pizza t...,2012
3,4ykgzzzGEWjMD5lwk1-l9A,2009-05-20,20,5,3,mixed results would definitely recommend buyin...,2009
4,2PfavOTufsPCRdYm-bFcpw,2012-08-01,1,8,5,place great ice cream shop favorite birch beer...,2012
5,8tBfgYCI6Fz9xrSYCYCYXA,2009-05-21,21,5,4,atmosphere place leave something desired alway...,2009
6,DhwPdZ-67IgFc_XcVS30BA,2009-05-21,21,5,4,years ago owner would sell pocket sandwiches p...,2009
7,AlvGCHZX0n5puR9eZSZ4lg,2009-05-21,21,5,2,burgers pricey nothing special fries hot cold ...,2009
8,zMJM_bYGj8x2ewzNfMRUkA,2012-10-31,31,10,4,boondas today pizza bacon really pretty good s...,2012
9,5UmKMjUEUNdYWqANhGckJw,2014-02-13,13,2,5,excellent food superb customer service miss ma...,2014


In [81]:
rev_and_business = pd.merge(reviews_info, business_info, how='inner', on=['business_id', 'business_id'])
print rev_and_business.shape
rev_and_business.head(10)

(2225213, 11)


,business_id,date,day,month,stars,text_clean,year,business_name,business_stars,city,state
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,1,8,4,mr hoagie institution walking seem like throwb...,2012,Mr Hoagie,4.5,Dravosburg,PA
1,5UmKMjUEUNdYWqANhGckJw,2014-02-13,13,2,5,excellent food superb customer service miss ma...,2014,Mr Hoagie,4.5,Dravosburg,PA
2,5UmKMjUEUNdYWqANhGckJw,2015-10-31,31,10,5,yes place little dated opened weekend staff al...,2015,Mr Hoagie,4.5,Dravosburg,PA
3,WaHXyBwljbKNPmmJZn5j8Q,2012-08-01,1,8,3,shop save,2012,Shop 'n Save,3.0,West Mifflin,PA
4,WaHXyBwljbKNPmmJZn5j8Q,2014-07-26,26,7,3,living alone love deals meat also love faygo p...,2014,Shop 'n Save,3.0,West Mifflin,PA
5,WaHXyBwljbKNPmmJZn5j8Q,2014-08-22,22,8,3,store changed hands many times past years show...,2014,Shop 'n Save,3.0,West Mifflin,PA
6,u22QDAON1kk0wjoTliyftw,2012-08-05,5,8,4,admit first time hate absolutely hated pizza t...,2012,Latina Pizza,4.0,West Mifflin,PA
7,u22QDAON1kk0wjoTliyftw,2008-04-05,5,4,4,underated ignored see ol time pizzaria near ke...,2008,Latina Pizza,4.0,West Mifflin,PA
8,u22QDAON1kk0wjoTliyftw,2013-08-18,18,8,4,good pizza great price unique crust crispy air...,2013,Latina Pizza,4.0,West Mifflin,PA
9,4ykgzzzGEWjMD5lwk1-l9A,2009-05-20,20,5,3,mixed results would definitely recommend buyin...,2009,Big Bang Bicycles,4.5,West Mifflin,PA


In [82]:
rev_and_business.to_pickle('rev_and_business')

In [73]:
print rev_and_business[1:10]
print rev_and_business['text_clean'].size

              business_id       date  stars  \
1  5UmKMjUEUNdYWqANhGckJw 2014-02-13      5   
2  5UmKMjUEUNdYWqANhGckJw 2015-10-31      5   
3  WaHXyBwljbKNPmmJZn5j8Q 2012-08-01      3   
4  WaHXyBwljbKNPmmJZn5j8Q 2014-07-26      3   
5  WaHXyBwljbKNPmmJZn5j8Q 2014-08-22      3   
6  u22QDAON1kk0wjoTliyftw 2012-08-05      4   
7  u22QDAON1kk0wjoTliyftw 2008-04-05      4   
8  u22QDAON1kk0wjoTliyftw 2013-08-18      4   
9  4ykgzzzGEWjMD5lwk1-l9A 2009-05-20      3   

                                          text_clean      business_name  \
1  excellent food superb customer service miss ma...          Mr Hoagie   
2  yes place little dated opened weekend staff al...          Mr Hoagie   
3                                          shop save       Shop 'n Save   
4  living alone love deals meat also love faygo p...       Shop 'n Save   
5  store changed hands many times past years show...       Shop 'n Save   
6  admit first time hate absolutely hated pizza t...       Latina Pizza   
7  u

In [5]:
reviews = pd.read_pickle('rev_and_business')
#reviews = rev_and_business
my_city = 'Las Vegas'

print 'FINDING TOP WORDS in', my_city

reviews_subset = reviews[reviews['city'] == my_city]

FINDING TOP WORDS in Las Vegas


In [6]:
reviews_random = reviews[reviews['day']%20 == 0]
print reviews_random.shape

(73984, 11)


In [7]:
reviews_random.to_pickle('reviews_random')

In [8]:
reviews_random.city.unique()

array(['West Mifflin', 'Pittsburgh', 'Homestead', 'rankin', 'Munhall',
       'West Homestead', 'Pittsburgh/Waterfront', 'Braddock',
       'Lower Lawrenceville', 'Dravosburg', 'Scottsdale', 'Las Vegas',
       'Carnegie', 'Sharpsburg', 'Bellevue', 'Edgewood', 'Mc Kees Rocks',
       'Mount Lebanon', 'Phoenix', 'Mckees Rocks', 'Bridgeville',
       'Lawrenceville', 'Green Tree', 'Charlotte', 'Dormont', 'Downtown',
       'Montr\xc3\xa9al', 'Madison', 'Mount Washington',
       'Pittsburgh/S. Hills Galleria', 'Heidelberg', 'Tempe', 'Aspinwall',
       'Bethel Park', 'Millvale', 'Mesa', 'Chandler', 'Cave Creek',
       'Gilbert', 'Matthews', 'Avondale', 'Peoria', 'Goodyear',
       'Litchfield Park', 'Henderson', 'Whitehall', 'Paradise Valley',
       'Glendale', 'Pineville', 'Shadyside', 'Casa Grande', 'Brentwood',
       'Concord', 'Champaign', 'Kitchener', 'Waterloo', 'Surprise',
       'Wickenburg', 'Edinburgh', 'Dorval', 'Robinson Township',
       'Paradise', 'New River', 'Verona',